* https://stackoverflow.com/questions/42471523/how-can-i-generate-a-proper-mnist-image
* https://stackoverflow.com/questions/45539289/convert-image-from-28-28-4-to-2d-flat-array-and-write-to-csv
* https://stackoverflow.com/questions/61552402/if-image-has-28-28-3-shape-how-do-i-convert-it-to-28-28-1
* https://stackoverflow.com/questions/51205502/convert-a-black-and-white-image-to-array-of-numbers

In [2]:
! conda install xgboost -y

Solving environment: done

## Package Plan ##

  environment location: /Users/champions2019/opt/anaconda3

  added / updated specs:
    - xgboost


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    _py-xgboost-mutex-2.0      |            cpu_0           8 KB
    ca-certificates-2022.4.26  |       hecd8cb5_0         124 KB
    certifi-2022.5.18.1        |   py39hecd8cb5_0         148 KB
    libxgboost-1.5.0           |       he9d5cce_1         1.2 MB
    openssl-1.1.1o             |       hca72f7f_0         2.2 MB
    py-xgboost-1.5.0           |   py39hecd8cb5_1         166 KB
    xgboost-1.5.0              |   py39hecd8cb5_1          25 KB
    ------------------------------------------------------------
                                           Total:         3.9 MB

The following NEW packages will be INSTALLED:

  _py-xgboost-mutex  pkgs/main/osx-64::_py-xgboost-mutex-2.0-cpu_0
  libxgb

In [3]:
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_openml

/Users/champions2019/opt/anaconda3/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [9]:
# import the mnist datsaet

mnist = fetch_openml('mnist_784', version=1)
mnist.keys()

dict_keys(['data', 'target', 'frame', 'categories', 'feature_names', 'target_names', 'DESCR', 'details', 'url'])

In [10]:
# separate features and target
X, y = mnist["data"], mnist["target"]

In [11]:
# train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                       test_size=0.2, 
                                       random_state=42)

In [32]:
# standardize
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

## Random Forest

In [33]:
# modeling: random forest (arbitrary hyperparameters)
rf = RandomForestClassifier(max_depth=8, min_samples_leaf=10, n_estimators=100)
rf.fit(X_train_scaled, y_train)

RandomForestClassifier(max_depth=8, min_samples_leaf=10)

In [34]:
# predict
y_preds=rf.predict(X_test_scaled)
print(list(y_preds[:10]))
print(list(y_test[:10]))

['8', '4', '8', '7', '7', '0', '6', '2', '7', '9']
['8', '4', '8', '7', '7', '0', '6', '2', '7', '4']


In [35]:
# evaluate
print(metrics.accuracy_score(y_test, y_preds))
print(metrics.precision_score(y_test, y_preds,average='macro'))
print(metrics.recall_score(y_test, y_preds,average='macro'))
print(metrics.f1_score(y_test, y_preds,average='macro'))

0.9246428571428571
0.9252220868700481
0.9237204888477797
0.9242368733924676


## XG Boost

There are in general two ways that you can control overfitting in XGBoost:

- The first way is to directly control model complexity.

    - This includes max_depth, min_child_weight and gamma.

- The second way is to add randomness to make training robust to noise.

    - This includes subsample and colsample_bytree.

    - You can also reduce stepsize eta. Remember to increase num_round when you do so.

[source](https://xgboost.readthedocs.io/en/stable/tutorials/param_tuning.html#:~:text=There%20are%20in,you%20do%20so.)

In [ ]:
# modeling: random forest (arbitrary hyperparameters)
rf = XGBClassifier(max_depth=6, min_child_weight=1, gamma=0, subsample=1, learning_rate=0.3)
rf.fit(X_train_scaled, y_train)

[10:48:03] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
